<a href="https://colab.research.google.com/github/eugenie-kim012/recommendation_system/blob/main/D%2B72_%EC%B6%94%EC%B2%9C01_%EC%9E%85%EB%AC%B8%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) 필요한 모듈을 import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print('⏩⏩')

2) Load 데이터

In [ ]:
# dataset을 노드에서 다운받아서 넣어줘야합니다!

import os
csv_path = './movie_dataset.csv'
df = pd.read_csv(csv_path)
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


3) 특성 선택

In [ ]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [ ]:
features = ['keywords','cast','genres','director']
features

['keywords', 'cast', 'genres', 'director']

In [ ]:
# 각 행(row)에 대해 여러 열의 문자열 데이터를 결합하는 함수 정의
def combine_features(row):
    # 'keywords', 'cast', 'genres', 'director' 열의 값을 공백(" ")으로 이어붙여 반환
    return row['keywords'] + " " + row['cast'] + " " + row['genres'] + " " + row['director']

# 데이터프레임 df의 처음 5개 행에 대해 combine_features 함수를 적용
combine_features(df[:5])

,0
0,culture clash future space war space colony so...
1,ocean drug abuse exotic island east india trad...
2,spy based on novel secret agent sequel mi6 Dan...
3,dc comics crime fighter terrorist secret ident...
4,based on novel mars medallion space travel pri...


In [1]:
# 결합에 사용할 열 이름들을 리스트로 정의 (예: features = ['keywords', 'cast', 'genres', 'director'])
for feature in features:
    # 각 열의 결측값(NaN)을 빈 문자열('')로 채움
    # 결측값이 있으면 문자열 결합 시 오류가 발생할 수 있기 때문에 미리 처리
    df[feature] = df[feature].fillna('')

# 각 행(row)에 대해 combine_features 함수를 적용하여 새로운 열 'combined_features' 생성
# combine_features 함수는 'keywords', 'cast', 'genres', 'director' 값을 공백으로 이어 붙여 하나의 문자열로 만듦
df["combined_features"] = df.apply(combine_features, axis=1)

# 생성된 'combined_features' 열 출력
df["combined_features"]

NameError: name 'features' is not defined

4) 벡터화하고 코사인 유사도를 계산

- 이제 이 칼럼을 벡터화하고, 코사인 유사도를 계산해보겠습니다. 장르, 배우명, 감독명의 텍스트 데이터를 범주형 데이터로 보기 때문에 단순하게 등장횟수를 세어 숫자 벡터로 만들겠습니다. 사이킷런의 CountVectorizer()를 사용하면 편하게 하실 수 있습니다.

In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])
print(type(count_matrix))
print(count_matrix.shape)
print(count_matrix)

<class 'scipy.sparse._csr.csr_matrix'>
(4803, 14845)
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 97547 stored elements and shape (4803, 14845)>
  Coords	Values
  (0, 3115)	1
  (0, 2616)	1
  (0, 4886)	1
  (0, 12386)	2
  (0, 14235)	1
  (0, 2755)	1
  (0, 12299)	1
  (0, 11517)	1
  (0, 14561)	1
  (0, 14820)	1
  (0, 11490)	1
  (0, 12134)	1
  (0, 14291)	1
  (0, 12567)	1
  (0, 7496)	1
  (0, 8831)	1
  (0, 11217)	1
  (0, 86)	1
  (0, 144)	1
  (0, 4435)	1
  (0, 11745)	1
  (0, 4566)	1
  (0, 6542)	1
  (0, 2061)	1
  (1, 86)	1
  :	:
  (4801, 10069)	1
  (4801, 5844)	1
  (4801, 252)	1
  (4801, 4098)	1
  (4801, 14796)	1
  (4801, 11361)	1
  (4801, 2978)	1
  (4801, 12036)	1
  (4801, 6138)	1
  (4802, 9659)	1
  (4802, 3812)	1
  (4802, 1788)	2
  (4802, 4210)	1
  (4802, 5181)	1
  (4802, 2912)	1
  (4802, 3821)	1
  (4802, 1069)	1
  (4802, 11185)	1
  (4802, 3681)	1
  (4802, 5399)	1
  (4802, 3894)	1
  (4802, 2056)	1
  (4802, 3093)	1
  (4802, 4502)	1
  (4802, 5900)	2


In [ ]:
# count_matrix : 각 영화(또는 데이터 포인트)를 벡터 형태로 표현한 희소 행렬 (예: CountVectorizer 또는 TfidfVectorizer 결과물)
# cosine_similarity : 두 벡터 간의 코사인 유사도를 계산 (0~1 사이 값, 1에 가까울수록 유사함)
cosine_sim = cosine_similarity(count_matrix)  # 각 영화 간 코사인 유사도를 계산하여 (N x N) 유사도 행렬 생성

# 유사도 행렬 출력
# 각 행과 열은 영화(또는 데이터 포인트)를 나타내며, 값은 두 영화 간 코사인 유사도
print(cosine_sim)  # 예: cosine_sim[0][1]은 0번째 영화와 1번째 영화의 유사도

# 유사도 행렬의 형태 출력
# (N, N) 형태 => N은 데이터(영화)의 개수, 즉 count_matrix의 행 개수
print(cosine_sim.shape)  # 예: (5000, 5000) => 5000개의 영화가 있으면 5000x5000 유사도 행렬


[[1.         0.10540926 0.12038585 ... 0.         0.         0.        ]
 [0.10540926 1.         0.0761387  ... 0.03651484 0.         0.        ]
 [0.12038585 0.0761387  1.         ... 0.         0.11145564 0.        ]
 ...
 [0.         0.03651484 0.         ... 1.         0.         0.04264014]
 [0.         0.         0.11145564 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.04264014 0.         1.        ]]
(4803, 4803)


5) 추천

In [ ]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

movie_user_likes = "Avatar"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index]))

sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

i=0
print(movie_user_likes+"와 비슷한 영화 3편은 "+"\n")
for item in sorted_similar_movies:
    print(get_title_from_index(item[0]))
    i=i+1
    if i==3:
        break

Avatar와 비슷한 영화 3편은 

Guardians of the Galaxy
Aliens
Star Wars: Clone Wars: Volume 1


In [ ]:
#타이타닉과 비슷한 영화 5편 뽑기

def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

movie_user_likes = "Titanic"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index]))

sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

i=0
print(movie_user_likes+"와 비슷한 영화 5편은 "+"\n")
for item in sorted_similar_movies:
    print(get_title_from_index(item[0]))
    i=i+1
    if i==5:
        break

# 행렬 인수분해

In [ ]:
import numpy as np
from numpy.linalg import svd

In [ ]:
np.random.seed(30)
A = np.random.randint(0, 100, size=(4, 4))
A

array([[37, 37, 45, 45],
       [12, 23,  2, 53],
       [17, 46,  3, 41],
       [ 7, 65, 49, 45]])

In [ ]:
svd(A)

SVDResult(U=array([[-0.54937068, -0.2803037 , -0.76767503, -0.1740596 ],
       [-0.3581157 ,  0.69569442, -0.13554741,  0.60777407],
       [-0.41727183,  0.47142296,  0.28991733, -0.72082768],
       [-0.6291496 , -0.46389601,  0.55520257,  0.28411509]]), S=array([142.88131188,  39.87683209,  28.97701433,  14.97002405]), Vh=array([[-0.25280963, -0.62046326, -0.4025583 , -0.6237463 ],
       [ 0.06881225, -0.07117038, -0.8159854 ,  0.56953268],
       [-0.73215039,  0.61782756, -0.23266002, -0.16767299],
       [-0.62873522, -0.47775436,  0.34348792,  0.50838848]]))

In [ ]:
# 결과값으로 행렬 U와 행렬 Σ, 행렬 V의 전치행렬이 생성됩니다. 우리는 이 값들을 unpacking해서 각각 변수명에 할당해 줍니다. 각 행렬들을 확인해 봅시다.

U, Sigma, VT = svd(A)
print('U matrix: {}\n'.format(U.shape),U)
print('Sigma: {}\n'.format(Sigma.shape),Sigma)
print('V Transpose matrix: {}\n'.format(VT.shape),VT)

U matrix: (4, 4)
 [[-0.54937068 -0.2803037  -0.76767503 -0.1740596 ]
 [-0.3581157   0.69569442 -0.13554741  0.60777407]
 [-0.41727183  0.47142296  0.28991733 -0.72082768]
 [-0.6291496  -0.46389601  0.55520257  0.28411509]]
Sigma: (4,)
 [142.88131188  39.87683209  28.97701433  14.97002405]
V Transpose matrix: (4, 4)
 [[-0.25280963 -0.62046326 -0.4025583  -0.6237463 ]
 [ 0.06881225 -0.07117038 -0.8159854   0.56953268]
 [-0.73215039  0.61782756 -0.23266002 -0.16767299]
 [-0.62873522 -0.47775436  0.34348792  0.50838848]]


In [ ]:
Sigma_mat = np.diag(Sigma)

A_ = np.dot(np.dot(U, Sigma_mat), VT)
A_

array([[37., 37., 45., 45.],
       [12., 23.,  2., 53.],
       [17., 46.,  3., 41.],
       [ 7., 65., 49., 45.]])